In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup

from time import sleep

from datetime import datetime
import pickle
import os

from hab.airtable.airtable import AirtableHAB
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

In [2]:
today = datetime.now().strftime("%Y%m%d")
# print(today)

with open(file = "serpapi_key.txt", mode = "r") as file:
    api_key = file.read()
    
# print(api_key)

lista_puestos = pd.read_excel("Lista_Puestos.xlsx", header = None)

lista_puestos.columns = ["jobs"]

In [3]:
%%time

list_json = list()

os.chdir("json_searches_latam")

for f in os.listdir():
    
    with open(file = f, mode = "br") as file:
        list_json.append(pickle.load(file))
        
os.chdir("..")

CPU times: total: 1.23 s
Wall time: 10.7 s


In [4]:
%%time

df = pd.concat(objs = [pd.json_normalize(x["jobs_results"]) for x in list_json if "jobs_results" in x],
               axis = 0,
               ignore_index = True)

CPU times: total: 5.22 s
Wall time: 6.92 s


In [5]:
%%time

job_types = ["Full-time", "Part-time", "Contractor"]

df.columns = [x.split(".")[1] if "." in x else x for x in df.columns]

df.drop("salary", axis = 1, inplace = True)

df["date_posted"] = datetime.strptime("2023-06-23", "%Y-%m-%d").date()

df.dropna(subset = "description", inplace = True)

# Drops Duplicates
df = df.drop_duplicates("job_id").reset_index(drop = True) 

# Cleans "via"
df["via"] = df["via"].apply(lambda x : clean.clean_source(x))

# Cleans "location"
df["location"] = df["location"].apply(lambda x : clean.clean_location(x))

# Cleans "contract_type"
df["schedule_type"] = df["schedule_type"].apply(lambda x : clean.clean_contract_type(x) if x in job_types else x)

# Cleans "created_date"
df["posted_at"] = df["posted_at"].apply(lambda x : clean.transform_date(x))

df["date_posted"] = [clean.get_date(x, y) for x, y in df[["date_posted", "posted_at"]].values]

# Creates "tech_skills"
df["tech_skills"] = df["description"].apply(lambda x : clean.get_skills(x))

df.drop("posted_at", axis = 1, inplace = True)

df["date_posted"] = df["date_posted"].astype(str)

CPU times: total: 2min 42s
Wall time: 3min 37s


In [6]:
df_copy = clean.clean_column_names(df).copy()

In [7]:
hab = AirtableHAB(table_name = "LATAM 001")

{'table_name': 'LATAM 001', 'field_ids': {'job_id': 'fldC1hvxSFs3rVPhE', 'title': 'fldDDNNf8htERcNjR', 'group': 'fldZl3hnctSgp68m0', 'group_metrics': 'fld7U1ILmMenTcffB', 'company_name': 'fldmG53TFTd70ncOo', 'location': 'fldwpOfa0yBWMDGo5', 'source': 'fldems3dVOfVFFmxR', 'description': 'fldKsFlu6eA8oCdiA', 'country': 'fldO6cCCt2O1Cy9Km', 'date_posted': 'fldagcwdLgg5REnoS', 'salary': 'fld87tdvYbWSfSvZf', 'salary_range': 'fld7yfvJgj2bWcUso', 'salary_avg': 'fldwzuAztwBQ9t1jA', 'contract_type': 'fldzyXhtLyrMdydWS', 'experience': 'fldls8dxF91fr5o3T', 'remote_work': 'fldChaVYkXhGPDfP6', 'tech_skills': 'fldGXGL2iXHtgc79s'}}


In [8]:
df_copy = hab.fill_na(df_copy)

In [9]:
df_copy.shape

(67933, 15)

In [10]:
df_copy = df_copy[[x for x in df_copy.columns if x.startswith("fld")]]

In [11]:
df_copy = df_copy.sort_values("fldmG53TFTd70ncOo", ascending = True)

# df_copy.groupby("fldmG53TFTd70ncOo").count().sort_values("fldwpOfa0yBWMDGo5", ascending = False).index

In [12]:
%%time
hab.airtable_post(df_copy.iloc[:50_000, :])

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [43]:
df_copy2 = df.sort_values("company_name", ascending = True).iloc[50_000:, :].copy()
df_copy2 = clean.clean_column_names(df_copy2)
df_copy2 = hab.fill_na(df_copy2)
df_copy2 = df_copy2[[x for x in df_copy2.columns if x.startswith("fld")]]

In [44]:
df_copy2

,fldiwwUNhDKO1Ikfh,fldYxY8gTmocufrQB,fldGv6eimflZ2Hprz,fldWusXVxlnUKHupV,fldGudJQ8dEbpobTz,fldAJqCNKWZByXl52,fldEPYF2kizL3cx4C,fld5PEhRnV7lasWNC,fldBJClPFOTPxx3QH,fldJmtgweYgES4ZPi
33535,Ingeniero /a/ de Big Data 1626250208.104,Servicio de Empleo Comfama,"Medellín, Medellin, Antioquia, Colombia",AIRA,Misión:\nEmpresa solicita Profesional en Ingen...,eyJqb2JfdGl0bGUiOiJJbmdlbmllcm8gL2EvIGRlIEJpZy...,Full-time,N/A,2023-06-23,[]
33519,Ingeniero /a/ de Big Data 1626250208.104,Servicio de Empleo Comfama,"Medellín, Medellin, Antioquia, Colombia",AIRA,Misión:\nEmpresa solicita Profesional en Ingen...,eyJqb2JfdGl0bGUiOiJJbmdlbmllcm8gL2EvIGRlIEJpZy...,Full-time,N/A,2023-06-23,[]
43814,Desarrollador Angular 1626250208.49,Servicio de Empleo Comfama,"Medellín, Medellin, Antioquia, Colombia",LinkerJOB Colombia,Descripción de la Empresa:\nEl Servicio de Emp...,eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIEFuZ3VsYX...,Full-time,N/A,2023-06-23,"[Angular, Java, JavaScript]"
42484,Consultor/A En Sistemas 1626135446.28 [MM-077],Servicio de Empleo Comfama,Colombia,Jobrapido.com,**Descripción empresa**: El Servicio de Empleo...,eyJqb2JfdGl0bGUiOiJDb25zdWx0b3IvQSBFbiBTaXN0ZW...,Full-time,N/A,2023-06-09,[]
33489,Ingeniero/a de sistemas y base de datos junior...,Servicio de Empleo Comfama,"Medellín, Medellin, Antioquia, Colombia",AIRA,Misión:\n\nDescripción:\nEmpresa dedicada a la...,eyJqb2JfdGl0bGUiOiJJbmdlbmllcm8vYSBkZSBzaXN0ZW...,Full-time,N/A,2023-06-23,[SQL]
...,...,...,...,...,...,...,...,...,...,...
52630,java开发工程师,川恒股份,Dominican Republic,LinkedIn República Dominicana,职位来源于智联招聘。\n\n岗位职责：\n\n1、负责公司软件项目的设计、开发、调试和维护；...,eyJqb2JfdGl0bGUiOiJqYXZh5byA5Y+R5bel56iL5biIIi...,Full-time,N/A,2023-06-23,[]
52947,Java开发工程师,软通动力信息技术(集团)股份有限公司,Dominican Republic,LinkedIn República Dominicana,职位来源于智联招聘。\n\n1、大专以上学历，计算机及相关专业；\n\n2、熟悉spring...,eyJqb2JfdGl0bGUiOiJKYXZh5byA5Y+R5bel56iL5biIIi...,Full-time,N/A,2023-06-23,[]
52632,Java开发工程师,软通动力信息技术(集团)股份有限公司,Dominican Republic,LinkedIn República Dominicana,职位来源于智联招聘。\n\n1、大专以上学历，计算机及相关专业；\n\n2、熟悉spring...,eyJqb2JfdGl0bGUiOiJKYXZh5byA5Y+R5bel56iL5biIIi...,Full-time,N/A,2023-06-23,[]
52949,java开发工程师,郑州思博科信息技术有限公司,Dominican Republic,LinkedIn República Dominicana,职位来源于智联招聘。\n\n1.5年及以上Web Java开发经验，能独立完成公司的开发项目...,eyJqb2JfdGl0bGUiOiJqYXZh5byA5Y+R5bel56iL5biIIi...,Full-time,N/A,2023-06-23,[]


In [45]:
%%time

hab = AirtableHAB(table_name = "LATAM 002")
hab.airtable_post(df_copy2)

{'table_name': 'LATAM 002', 'field_ids': {'job_id': 'fldAJqCNKWZByXl52', 'title': 'fldiwwUNhDKO1Ikfh', 'group': 'fldSR7hZqldnrtjty', 'group_metrics': 'fldiyQN1WI2Ep0JWu', 'company_name': 'fldYxY8gTmocufrQB', 'location': 'fldGv6eimflZ2Hprz', 'source': 'fldWusXVxlnUKHupV', 'description': 'fldGudJQ8dEbpobTz', 'country': 'fldiW7kPbIfSzMjw4', 'date_posted': 'fldBJClPFOTPxx3QH', 'salary': 'fldJwESWgF2wOginv', 'salary_range': 'fldhJ1c4157Gihr5Q', 'salary_avg': 'fldDhgT2RWBsIn1OW', 'contract_type': 'fldEPYF2kizL3cx4C', 'experience': 'fldlaz4yR3bLqXOWz', 'remote_work': 'fld5PEhRnV7lasWNC', 'tech_skills': 'fldJmtgweYgES4ZPi'}}
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200

In [55]:
import re

def contiene_numero(cadena):
    
    patron = r"\d+"
    
    if re.search(patron, cadena):
        return True
    else:
        return False

In [54]:
df["description"].apply(lambda x : contiene_numero(x)).value_counts(normalize = True)

True     0.847821
False    0.152179
Name: description, dtype: float64

In [14]:
################################################################################################################################